# Preparation

In [ ]:
import bert-serving-server
import install bert-serving-client
import install tensorflow

In [13]:
# read data
import json
json_data = open("/Users/arlenehuang/OneDrive - Columbia University/Summer 2020/APAN 5430/Assignment 7/webhose_Citigroup_deduplicated.json").readlines()
newsfeeds = []

for line in json_data:
    newsfeeds.append(json.loads(line))

In [15]:
# get titles
titles = []
for line in newsfeeds:
    titles.append(line['title'])

# Part I Implements a keyword or sentence taxonomy of topic classes

In [1]:
topic_taxonomy = {
    "Ratings":
    {
        "neutral rating": "A neutral rating is a stock analyst rating that is neither extremely negative nor extremely positive; instead, the stock is expected to perform in line with the expected returns of the market. It is mostly the same as hold rating.",
        "rating hold": "Hold is an analyst's recommendation to neither buy nor sell a security. This rating is better than sell but worse than buy, meaning that investors with existing long positions shouldn't sell but investors without a position shouldn't purchase either.",
        "rating buy": "A buy rating is an investment analyst's recommendation to buy a security, and implies the security is undervalued.",
        "rating sell": "A sell rating is an investment analyst's recommendation to sell a security, and implies the security is overvalued."
    },
    "Stock Market":
    {
        "hedge fund": "A hedge fund is an aggressively managed portfolio of investments that uses leveraged, long, short and derivative positions.",
        "price target": "A price target is an analyst's projection of a security's future price.",
        "stock price": "Stock price is the price of common shares for a given company, in this case, it is the price of Citigroups' stock",
        "day move average": "The day moving average (DMA) is a simple technical analysis tool that smooths out price data by creating a constantly updated average price by day"
    },
    "Evaluation":
    {
        "quick ratio": "The quick ratio is an indicator of a company's short-term liquidity position and measures a company's ability to meet its short-term obligations with its most liquid assets. ",
        "current ratio": "The current ratio is a liquidity ratio that measures a company's ability to pay short-term obligations or those due within one year. It tells investors and analysts how a company can maximize the current assets on its balance sheet to satisfy its current debt and other payables.",
        "equity ratio": "The equity ratio is a financial metric that measures the amount of leverage used by a company. ",
        "equity return": "Return on equity (ROE) is a measure of financial performance calculated by dividing net income by shareholders' equity. Because shareholders' equity is equal to a company's assets minus its debt, ROE is considered the return on net assets.",
        "net margin": "Net margin is the percentage of revenue remaining after all operating expenses, interest, taxes and preferred stock dividends have been deducted from a company's total revenue.",
        "quarter revenue": "Quarterly revenue is the revenue of a quarter, it helps analyst to measure the increase in a firm's sales from one quarter to another, which is more precise."
    } 
}

In [5]:
import json
with open("/Users/arlenehuang/OneDrive - Columbia University/Summer 2020/APAN 5430/Assignment 9/taxonomy.json", "w") as data_file:
    json.dump(topic_taxonomy, data_file)

In [71]:
defi_list = []
tax_list = []
for key, value in topic_taxonomy.items():
    for label, keywords in value.items():
        defi_list.append(keywords.lower())
        tax_list.append(label)
topic_list = []
for topic in topic_taxonomy:
    topic_list.append(topic)

# Part II Classifies your Webhose article titles against the developed taxonomy using semantic_text_similarity package and groups them by the topic classes

#### Calculate similarities of whole dataset

In [72]:
from semantic_text_similarity.models import WebBertSimilarity

model = WebBertSimilarity(device='cpu', batch_size=10) #defaults to GPU prediction

In [141]:
def cal_similarity(sent):
    result = []
    for i in range(14):
        result.append(model.predict([(sent,defi_list[i])])[0])
    return result
def get_topic(sent):
    similarity = cal_similarity(sent)
    tax_index = similarity.index(max(similarity))
    if tax_index <= 4:
        topic_index = 0
    elif tax_index <= 8:
        topic_index = 1
    elif tax_index <= 14:
        topic_index = 2
    tax = tax_list[tax_index]
    topic = topic_list[topic_index]
    max_simi = max(similarity)
    return tax, topic, max_simi

In [143]:
import pandas as pd
df = pd.DataFrame(columns = ['title','topic','taxonomy', 'similarity'])
for i in range(len(titles)):
    a = [titles[i]]
    result = get_topic(titles[i])
    a.append(result[0])
    a.append(result[1])
    a.append(result[2])
    df.loc[i] = a
df

,title,topic,taxonomy,similarity
0,It’s like the Wild West in the stock market wi...,stock price,Stock Market,1.233517
1,Esperion Therapeutics’ (ESPR) Buy Rating Reaff...,rating buy,Ratings,1.944404
2,Why Citi is the latest Wall Street giant to ba...,stock price,Stock Market,1.369271
3,Centene (NYSE:CNC) Updates FY20 Earnings Guidance,day move average,Stock Market,1.065095
4,CVS Health (NYSE:CVS) Issues FY 2020 After-Hou...,equity return,Evaluation,0.922252
...,...,...,...,...
3308,Kingfisher KGF) Stock Rating Upgraded by Morga...,neutral rating,Ratings,1.553288
3309,Tupperware Brands Co. (NYSE:TUP) Sees Signific...,quick ratio,Stock Market,1.175745
3310,Cactus (NYSE:WHD) Stock Rating Lowered by Barc...,neutral rating,Ratings,1.739906
3311,Bankers face ‘new normal' as they return to of...,rating buy,Ratings,0.914969


In [162]:
# made a mistake in creating df column names
# correct column names
df.columns = ['title','taxonomy','topic', 'similarity']
df

,title,taxonomy,topic,similarity
0,It’s like the Wild West in the stock market wi...,stock price,Stock Market,1.233517
1,Esperion Therapeutics’ (ESPR) Buy Rating Reaff...,rating buy,Ratings,1.944404
2,Why Citi is the latest Wall Street giant to ba...,stock price,Stock Market,1.369271
3,Centene (NYSE:CNC) Updates FY20 Earnings Guidance,day move average,Stock Market,1.065095
4,CVS Health (NYSE:CVS) Issues FY 2020 After-Hou...,equity return,Evaluation,0.922252
...,...,...,...,...
3308,Kingfisher KGF) Stock Rating Upgraded by Morga...,neutral rating,Ratings,1.553288
3309,Tupperware Brands Co. (NYSE:TUP) Sees Signific...,quick ratio,Stock Market,1.175745
3310,Cactus (NYSE:WHD) Stock Rating Lowered by Barc...,neutral rating,Ratings,1.739906
3311,Bankers face ‘new normal' as they return to of...,rating buy,Ratings,0.914969


In [235]:
# store the data incase accidentally delete it
store = df

#### identify top 10 ariticle based on topics

In [237]:
# for topics
def get_topX_articles_topic(X=10):
    result = pd.DataFrame(columns = ['title','topic','taxonomy', 'similarity'])
    for topic in topic_list:
        sub_df = df[df["topic"] == topic]
        sub_df = pd.DataFrame(sub_df)
        sub_df.sort_values("similarity", ascending = False, inplace = True)
        result_append = sub_df.head(X)
        result = result.append(result_append)
    return result

In [238]:
result_topics = get_topX_articles_topic(10)
result_topics

,title,topic,taxonomy,similarity
2095,State Street Corp (NYSE:STT) Receives Consensu...,Ratings,rating hold,2.822666
618,Eni SpA (NYSE:E) Given Consensus Recommendatio...,Ratings,rating hold,2.798796
2358,Dell Inc. (NYSE:DELL) Given Consensus Rating o...,Ratings,rating hold,2.654552
496,Entergy Co. (NYSE:ETR) Receives Consensus Rati...,Ratings,rating buy,2.638032
1418,Extended Stay America (NYSE:STAY) Receives Con...,Ratings,rating hold,2.634744
2530,General Dynamics (NYSE:GD) Earns Buy Rating fr...,Ratings,rating buy,2.633881
1422,Stag Industrial Inc (NYSE:STAG) Given Consensu...,Ratings,rating buy,2.575736
2148,"Advanced Energy Industries, Inc. (NASDAQ:AEIS)...",Ratings,rating hold,2.560562
2091,TJX Companies Inc (NYSE:TJX) Given Consensus R...,Ratings,rating buy,2.539830
2625,"Amkor Technology, Inc. (NASDAQ:AMKR) Receives ...",Ratings,rating hold,2.539368


#### identify top 10 ariticle based on taxonomy

In [239]:
# for taxonomy
def get_topX_articles_tax(X=10):
    result = pd.DataFrame(columns = ['title','topic','taxonomy', 'similarity'])
    for tax in tax_list:
        sub_df = df[df["taxonomy"] == tax]
        sub_df = pd.DataFrame(sub_df)
        sub_df.sort_values("similarity", ascending = False, inplace = True)
        result_append = sub_df.head(X)
        result = result.append(result_append)
    return result

In [240]:
result_tax = get_topX_articles_tax(10)
result_tax

,title,topic,taxonomy,similarity
2741,Somewhat Positive Press Coverage Extremely Lik...,Ratings,neutral rating,2.536175
681,United Utilities Group’s (UU) Neutral Rating R...,Ratings,neutral rating,2.293071
1725,Trade Desk (NASDAQ:TTD) Stock Rating Reaffirme...,Ratings,neutral rating,2.170673
568,Intermediate Capital Group (LON:ICP) Stock Rat...,Ratings,neutral rating,2.159617
1728,Passage Bio (NASDAQ:ORIC) Stock Rating Lowered...,Ratings,neutral rating,2.041143
...,...,...,...,...
2511,Expedia Group (NASDAQ:EXPE) Shares Gap Up on A...,Evaluation,quarter revenue,1.739880
1818,Qudian (NYSE:QD) Posts Quarterly Earnings Resu...,Evaluation,quarter revenue,1.686690
1858,FY2020 EPS Estimates for Stag Industrial Inc R...,Evaluation,quarter revenue,1.668005
2023,Qudian (NYSE:QD) Announces Quarterly Earnings ...,Evaluation,quarter revenue,1.615380


In [241]:
# Explaination:
# Total rows = 134 is becaue "current ratio" is not common in titles
# and it only show up 4 times in the dataset
# "current ratio" needs to be kept in dataset since it is very important
# and highly related
pd.value_counts(result["taxonomy"])

net margin          10
equity return       10
quarter revenue     10
hedge fund          10
rating hold         10
price target        10
stock price         10
quick ratio         10
neutral rating      10
rating sell         10
equity ratio        10
day move average    10
rating buy          10
current ratio        4
Name: taxonomy, dtype: int64